In [1]:
import os
import sys
import ast
import cv2
import json
import math
import shutil
import random
import pickle
import imageio
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
sys.path.append('./')
from utils import normalize_image, plot_image, plot_images

In [3]:
# load human pose info
human_pose_csv = '/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/all_human_pose.csv'
hp_df = pd.read_csv(human_pose_csv, converters={"properties__json": ast.literal_eval})
print(hp_df.shape)
hp_df.head(2)

(122882, 3)


,id,image,properties__json
0,65c1834cbad271dacf97e0e7,5f81f1d7bf0c3674c05778c4,"[{'featureId': 'cls9tzeqs000a356l9v59fjwt', 'v..."
1,65c18282ba421ac96fb33c79,60679aa31e3d26cab0c8c694,"[{'featureId': 'clsa1ufc10006356ns6mt2ud2', 'v..."


In [2]:
# root_dir = '/data/jupiter/datasets/'
root_dir = '/data2/jupiter/datasets/'
# dataset = 'Jupiter_train_v6_2'
dataset = 'halo_rgb_stereo_train_v6_2'
# dataset = 'halo_rgb_stereo_test_v6_1'
csv = os.path.join(root_dir, dataset, 'master_annotations_dedup.csv')
converters = {"label_map": ast.literal_eval, "label_counts": ast.literal_eval}
df = pd.read_csv(csv, converters=converters)
df.shape

/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (2,3,24,26,82,93,94,95,96,97,99,100,102,103,104,105,106,107,108,109,110,120,151,153,157,158,163,172,175,178,181,182,184,188,190,200,201,203,204,205,206,211,228,233,234,235,236,238,240,241,242,253,254,256,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,277,282,283,284,287,288,289,291) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(280564, 348)